### Churn @ Robinhood
#### Load Libraries and Data

In [1]:
from numba import cuda

try:
    print("Available GPUs:", cuda.gpus)
except cuda.CudaSupportError as e:
    print("CUDA Error:", e)

Available GPUs: <Managed Device 0>


In [2]:
import cudf as cf
equity_df_raw = cf.read_csv('./data/equity_value_data.csv')
features_df_raw = cf.read_csv('./data/features_data.csv')

equity_df = equity_df_raw.copy()
features_df = features_df_raw.copy()

In [3]:
equity_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   timestamp     1119158 non-null  object
 1   close_equity  1119158 non-null  float64
 2   user_id       1119158 non-null  object
dtypes: float64(1), object(2)
memory usage: 72.6+ MB


In [4]:
features_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   risk_tolerance                5584 non-null   object
 1   investment_experience         5584 non-null   object
 2   liquidity_needs               5584 non-null   object
 3   platform                      5584 non-null   object
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object
 8   user_id                       5584 non-null   object
dtypes: float64(2), object(7)
memory usage: 896.8+ KB


#### a). What percentage of users have churned in the data?
A user is *churned* when their equity falls below 10 usd for 28 consecutive calendar days or longer having perviously been at least 10 usd

**NOTE** Since no equities falls under 10 usd, threshold is set to 11 usd instead

In [7]:
# Step 1: Ensure the timestamp column is datetime and sort the data
equity_df['timestamp'] = cf.to_datetime(equity_df['timestamp'])
equity_df = equity_df.sort_values(['user_id', 'timestamp'])

# Step 2: Flag close_equity below 11
equity_df['below_11'] = (equity_df['close_equity'] < 11).astype(int)

# Step 3: Compute rolling 28-day windows for each user
equity_df['below_11_28d'] = equity_df.groupby('user_id')['below_11'].rolling(window=28, min_periods=28).sum().reset_index(0, drop=True)

# Step 4: Identify churn (continuous 28 days below $11)
equity_df['churn'] = (equity_df['below_11_28d'] == 28).astype(int).copy()

# Step 5: Check if the user ever had close_equity >= 11
# Group by 'user_id' to find the max close_equity
user_max_equity = equity_df.groupby('user_id')['close_equity'].max().reset_index()
user_max_equity.rename(columns={"close_equity": "max_equity"}, inplace=True)

# Merge back to associate max_equity with each user_id in the main DataFrame
equity_df = equity_df.merge(user_max_equity, on="user_id", how="left")

# Add a flag for users who had close_equity >= 11 at some point
equity_df['above_11_before'] = (equity_df['max_equity'] >= 11).astype(int)

# Step 6: Filter churned users
churned_users = equity_df.loc[
    (equity_df['churn'] == 1) & (equity_df['above_11_before'] == 1),
    'user_id'
].unique()

# Step 7: Calculate the churn percentage
total_users = equity_df['user_id'].nunique()
churn_percentage = (len(churned_users) / total_users) * 100

print(f"Churned Percentage: {churn_percentage:.2f}%")

Churned Percentage: 0.88%
